In [1]:
!pip install transformers[sentencepiece]
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 28.1 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 6.6 MB 52.6 MB/s 
     |████████████████████████████████| 596 kB 54.0 MB/s 
     |████████████████████████████████| 1.2 MB 32.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 15.3 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
     |████████████████████████████████| 1.1 MB 39.6 MB/s 
     |████████████████████████████████| 212 kB 58.0 MB/s 
     |████████████████████████████████| 140 kB 54.5 MB/s 
     |████████████████████████████████| 127 kB 77.8 MB/s 
     |█████████████

In [2]:
import os
import numpy as np
import pandas as pd
import transformers
import torch
from transformers import TFAutoModel, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

In [3]:
MODEL_EMBED = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"

In [4]:
# Embed layer
tokenizer_EMBED = AutoTokenizer.from_pretrained(MODEL_EMBED)
model_EMBED = AutoModelForSequenceClassification.from_pretrained(MODEL_EMBED)

Downloading:   0%|          | 0.00/463 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [5]:
# body layer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer_BODY = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model_BODY = AutoModelForSeq2SeqLM.from_pretrained('mrm8488/t5-base-finetuned-question-generation-ap')

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [7]:
model_EMBED.deberta.embeddings.word_embeddings

Embedding(251000, 768, padding_idx=0)

In [6]:
model_EMBED

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0): DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
 

In [24]:
model_BODY.shared = model_EMBED.deberta
model_BODY.encoder.embed_tokens = model_BODY.shared
model_BODY.decoder.embed_tokens = model_BODY.shared
model_BODY.lm_head = torch.nn.Linear(in_features=768, out_features=251000, bias=False)

In [9]:
# model_BODY

In [10]:
from datasets import load_dataset

dataset = load_dataset("iapp_wiki_qa_squad")

Generating train split:   0%|          | 0/5761 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/742 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/739 [00:00<?, ? examples/s]

Dataset iapp_wiki_qa_squad downloaded and prepared to /root/.cache/huggingface/datasets/iapp_wiki_qa_squad/iapp_wiki_qa_squad/1.0.0/c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
train_dataset  = load_dataset('iapp_wiki_qa_squad', split="train")
valid_dataset = load_dataset('iapp_wiki_qa_squad', split="validation")
test_dataset = load_dataset("iapp_wiki_qa_squad", split="test")

Reusing dataset iapp_wiki_qa_squad (/root/.cache/huggingface/datasets/iapp_wiki_qa_squad/iapp_wiki_qa_squad/1.0.0/c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)
Reusing dataset iapp_wiki_qa_squad (/root/.cache/huggingface/datasets/iapp_wiki_qa_squad/iapp_wiki_qa_squad/1.0.0/c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)
Reusing dataset iapp_wiki_qa_squad (/root/.cache/huggingface/datasets/iapp_wiki_qa_squad/iapp_wiki_qa_squad/1.0.0/c1455d806e5a66ca9ee5c03b4aeaeaef4410afca6263c0bfb440ff1db28e20c3)


In [12]:
train_dataset

Dataset({
    features: ['question_id', 'article_id', 'title', 'context', 'question', 'answers'],
    num_rows: 5761
})

In [13]:
def edit_ans_to_ds(dataset):
#     ans_list = [ans_dict["text"][0] for ans_dict in dataset["answers"]]
    dataset["ans_list"] = dataset["answers"]["text"][0]
#     dataset.add_column("ans_list", ans_list)
    return dataset

In [14]:
train_dataset = test_dataset.map(edit_ans_to_ds)
train_dataset

  0%|          | 0/739 [00:00<?, ?ex/s]

Dataset({
    features: ['question_id', 'article_id', 'title', 'context', 'question', 'answers', 'ans_list'],
    num_rows: 739
})

In [15]:
def create_ans_context(dataset):
    answer = dataset["ans_list"]
    context = dataset["context"]
    input_format = tokenizer_EMBED("answer: %s  context: %s </s>" % (answer, context), truncation=True ,padding=True)
    labels = tokenizer_EMBED(dataset["question"],truncation=True ,padding=True)
    dataset["input_ids"] = input_format["input_ids"]
    dataset["attention_mask"] = input_format["attention_mask"]
    dataset["labels"] = labels["input_ids"]
    return dataset

In [16]:
train_ds_processed = train_dataset.map(create_ans_context)

  0%|          | 0/739 [00:00<?, ?ex/s]

In [17]:
train_ds_processed = train_ds_processed.remove_columns(['question_id', 'article_id', 'title', 'context', 'question', 'answers', 'ans_list'])

In [18]:
train_ds_processed.set_format("torch")

In [19]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_ds_processed, shuffle=True, batch_size=1)
# eval_dataloader = DataLoader(train_ds, batch_size=1)

In [20]:
from torch.optim import AdamW
optimizer = AdamW(model_BODY.parameters(), lr=5e-5)

In [21]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [22]:
# ls

In [23]:
model_BODY = model_BODY.load_tf_weights("MP")

TypeError: ignored

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_BODY.to(device)

In [ ]:
!nvidia-smi

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model_BODY.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model_BODY(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        running_loss += loss.item()
    print(epoch, running_loss)

In [ ]:
model_BODY

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_BODY.save_pretrained("/content/drive/MyDrive/weight")

In [ ]:
batch

In [ ]:
# !pip install 

In [ ]:
eval_dataloader = train_dataloader

In [ ]:
!pip install sacrebleu

In [ ]:
import datasets

In [ ]:
metric = datasets.load_metric("sacrebleu")
model_BODY.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model_BODY(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

In [ ]:
tokenizer_EMBED.decode(predictions[0])

In [ ]:
predictions